-----
# Causal Discovery Grounding: an Implementation

Code for the basic method with a basic simulator.

In [1]:
# Library loading

%load_ext autoreload
%autoreload 2
from simulator import *

from scipy.optimize import linprog
from tqdm import tqdm

synthetic_demo_1 = False
debug_bounds = False

--------
## Placeholder Background Tools

The following are simple implementations of conditional independence and simulators that should be substituted with better alternatives.

In [2]:
# Conditional independence test

from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

def regression_indep(df, x_idx, a_idx, y_idx):
    
  """
  Finds independence between covariates and output using sparse logistic regression.

  Parameters:
  -----------

  df    : pandas DataFrame
  x_idx : list of strings with the column names corresponding to the covariates 
  a_idx : string with the name of the (binary) treatment variable
  y_idx : string with the name of the (binary) outcome variable

  Returns:
  --------
    
  drop_Y : list of binary variables, drop_Y[i] == 1 indicates that variable
           correspoding to x_idx[i] has been dropped in the regression of outcome
           on covariates X and treament A
  drop_X : similar, but the regression is of the treatment A on covariantes X only
  """

  def get_indep_sel(model_reg, names_in):
      
    ohe = model_reg.named_steps["onehot"]
    feature_names = ohe.get_feature_names_out(names_in)
    coef = model_reg.named_steps["logreg"].coef_[0]

    indep_list = np.zeros(len(x_idx), dtype=np.int_)
    for j, x_name in enumerate(x_idx):
      indices = [i for i, s in enumerate(feature_names) if s.startswith(x_name)]
    if sum(coef[indices]==0) >= 0.5 * len(indices):
      indep_list[j] = 1

    return indep_list
    
  xa_idx = x_idx.copy()
  xa_idx.extend([a_idx])
  A = df[a_idx]
  Y = df[y_idx]
  X = df[x_idx]
  XA = df[xa_idx]

  model_reg = Pipeline([
      ("onehot", OneHotEncoder(drop="first", sparse_output=True)),
      ("logreg", LogisticRegression(
          penalty="l1",
          solver="saga",
          max_iter=5000,
          C=0.1       # smaller C → more sparsity
      ))
  ])

  model_reg.fit(XA, Y)
  drop_Y = get_indep_sel(model_reg, xa_idx)
  model_reg.fit(X, A)
  drop_A = get_indep_sel(model_reg, x_idx)

  return drop_Y, drop_A


In [3]:
# Model and data simulation

def simulate_plain_data_collection(k, n_obs, n_rct, num_x, p_sparse):

  """
  The plainest simulator: generate a bunch of binary models with independent covariates,
  with corresponding observational and RCT samples. The "prior" tying them together is
  a very broad one (the simulator inside `simulate_binary_synthetic_backdoor_parameters`), 
  so problems are not well-connected. True sparsity is imposed on the outcome model.

  Parameters:
  -----------

  k        : number of datasets
  n_obs    : sample size for observational datasets
  n_rct    : sample size for RCTs
  num_x    : number of covariates
  p_sparse : probability of dropping a covariate into the outcome model

  Returns:
  --------
    
  true_models : models of the type `BinarySyntheticBackdoorModel`
  dat_obs     : a Pandas data frame with the corresponding observational data
  dat_rct     : a Pandas data frame with the corresponding randomized trial
  true_cates  : true CATEs evaluated at the corresponding dat_obs datasets
  """

  sparse_pattern = np.random.binomial(n=1, p=p_sparse, size=num_x)

  true_models = [None] * k
  dat_obs = [None] * k
  dat_rct = [None] * k
  true_cates = [None] * k

  for i in range(k):
    true_models[i] = simulate_binary_synthetic_backdoor_parameters(num_x, sparse_pattern)
    dat_obs[i] = simulate_from_backdoor_model(n_obs, true_models[i])
    dat_rct[i] = simulate_from_controlled_backdoor_model(n_rct, true_models[i])
    true_cates[i] = get_cate(dat_obs[i], true_models[i])

  return true_models, dat_obs, dat_rct, true_cates


In [4]:
# ILLUSTRATIVE DATA SIMULATION
#
# Purely synthetic case where all variables are binary and 


if synthetic_demo_1:

  n = 1000       # Sample size to simulate a fitting procedure
  num_x = 20     # Number of covariates
  p_sparse = 0.2 # Probability of dropping a covariate from the outcome model
  sparse_pattern = np.random.binomial(n=1, p=p_sparse, size=num_x)

  # Generate synthetic data from fitted model, along with ground truth CATE
  print('Simulating ground truth model...')
  true_model = simulate_binary_synthetic_backdoor_parameters(num_x, sparse_pattern)
  model_sample = simulate_from_backdoor_model(n, true_model)
  true_cate = get_cate(model_sample, true_model)

  # Fit model with all covariates
  print('Fitting causal effect estimators...')
  cate_hat_full, _ = estimate_cate_tlearner(model_sample, true_model.x_idx, true_model)

  print()

  # Please notice that the CATE cases are not directly comparable, as the CATE changes between problems
  print('RESULTS FOR SYNTHETIC DATA EXPERIMENT')
  print('------------------------------------------')
  print('True CATE standard deviation:', np.mean(abs(true_cate - cate_hat_full)))
  print('Mean absolute CATE error:', np.mean(abs(true_cate - cate_hat_full)))
  print('ATE error:', np.mean(true_cate) - np.mean(cate_hat_full))
  print('CATE rank correlation:', spearmanr(true_cate, cate_hat_full).statistic)



----
## Main Algorithm

The following are the core functions for the discovery grounding algorithm.

In [5]:
# Methods for bounding inference

def search_adjustment(dat, x_idx, a_idx, y_idx):

  """
  Search for possible pseudo-intervention variable ("instruments") that can be found
  in observational data `dat`. This is done by running a conditional independence criteria
  that tests for independence of outcome and possible instrument Z_A given all other covariates
  and treatment variable, as well a conditional independence assessment of Z_A and treatment
  given all other covariates. If independence is detected in the first case but not the latter one,
  then Z_A is selected as a instrumental variable.
  
  Parameters:
  -----------

  dat   : observational data, a Pandas dataframe
  x_idx : list of strings with the column names corresponding to the covariates 
  a_idx : string with the name of the (binary) treatment variable
  y_idx : string with the name of the (binary) outcome variable

  Returns:
  --------

  instruments: a binary array indicating which variables passed the test,
                        where the positions correspond to positions in `x_idx`
  """

  drop_Y, drop_A = regression_indep(dat, x_idx, a_idx, y_idx)
  drop_in = (drop_Y == 1) * (drop_A == 0)
  instruments = [i for i, value in enumerate(drop_in) if value == 1]

  return instruments

def get_predictive_model(train_obs, train_rct, x_idx, a_idx, y_idx, instruments):
  
  """
  Fits models for predicting treatment, and outcome under fixed treatments.

  Parameters:
  -----------

  train_obs : pandas DataFrame collected under observational conditions
  train_rct : pandas DataFrame collected under controlled conditions
  x_idx     : list of strings with the column names corresponding to the covariates 
  a_idx     : string with the name of the (binary) treatment variable
  y_idx     : string with the name of the (binary) outcome variable
  instruments: subset of covariates (given as positions in `x_idx`) which
               should not be used for the outcome model

  Returns:
  --------
    
  model_a  : XGBoost model that predicts P(A = 1 | X = x)
  model_y1 : XGBoost model that predicts P(Y = 1 | X = x, A = 1)
  model_y0 : XGBoost model that predicts P(Y = 1 | X = x, A = 0)
  """
  
  model_a = learn_xgb(train_obs[x_idx], train_obs[a_idx])

  x_idx_in = [x for j, x in enumerate(x_idx) if j not in instruments]
  
  a_train = train_rct[a_idx]
  model_y1 = learn_xgb(train_rct[x_idx_in][a_train==1], train_rct[y_idx][a_train==1])
  model_y0 = learn_xgb(train_rct[x_idx_in][a_train==0], train_rct[y_idx][a_train==0])
  
  a_train = train_obs[a_idx]
  model_y1o = learn_xgb(train_obs[x_idx_in][a_train==1], train_obs[y_idx][a_train==1])
  model_y0o = learn_xgb(train_obs[x_idx_in][a_train==0], train_obs[y_idx][a_train==0])

  return model_a, model_y1, model_y0, model_y1o, model_y0o

def bound_po(model_ya, model_a, x_value, x_pos, x_space, instruments, eps, test_values=None):
   
   """
   Given a model for P(Y_a = 1 | X = `x_value`) and P(A = 1 | X = `x_value`), this returns lower and upper bounds
   on P(A = 1 | X = `x_value`, Y_a = 1). (Notice that "a" in {0, 1} is implicit here, given indirectly by 
   P(Y_a = 1 | X = `x_value') model). It does so by making use of X[`x_pos`] as an instrument, and assuming 
   particular constraints bounded by `eps`.

   Parameters:
   -----------

   model_ya : a model that returns P(Y_1 = 1 | X = x), a xgboost tree
   model_a  : a model that returns P(A = 1 | X = x), a xgboost tree
   x_value  : instance of covariates where we are evaluating the quantity of interest
   x_pos    : position in the covariate vector of the variable which is the inferred instrument
   x_space  : an integer indicating that X[x_pos] takes values in 0, 1, 2, ..., x_space - 1
   instruments: subset of covariates (given as positions in `x_value`) which
              should not be used for the outcome model
   eps      : corresponding bound to define constraints
   test_values: for debugging purposes, if true we put here the test values of the decision variables
                to check whether constrains hold

   Returns:
   --------
    
   l, u : lower and upper bounds, respectively on P(A = 1 | X = `x_value`, Y_a = 1)
   """
   
   xg = x_value[0, x_pos] # The value taken by the instrument with the respective level `x_value`.
   x_value_in = np.delete(x_value[0, :], instruments)[None, :]
  
   # We will now build a small linear program with 2 * x_space decision variables. The decision variables are
   # correspond to theta_1x := P(A = 1 | X[x_pos] = x, X[\x_pos] = x_value[\x_pos], Y_a = 1), and
   # theta_0x := P(A = 1 | X[x_pos] = x, X[\x_pos] = x_value[\x_pos], Y_a = 0), where \xpos means
   # "everything in range(x_space), but x_pos". This program will have x_space equality constraints, 
   # and 4 * (x_space - 1) inequality constraints.

   # Equality constraints: 
   # P(A = 1 | X[x_pos] = x, X[\x_pos] = x_value[\x_pos], Y_a = 1) * 
   #     P(Y_a = 1 | X[x_pos] = x, X[\x_pos] = x_value[\x_pos]) +
   # P(A = 1 | X[x_pos] = x, X[\x_pos] = x_value[\x_pos], Y_a = 0) * 
   #     P(Y_a = 0 | X[x_pos] = x, X[\x_pos] = x_value[\x_pos]) = 
   # P(A = 1 | X[x_pos] = x, X[\x_pos] = x_value[\x_pos])
   
   A_eq = np.zeros([x_space, 2 * x_space])
   b_eq = np.zeros(x_space)
   for x in range(x_space):
     x_value_x = x_value.copy()
     x_value_x[0, x_pos] = x
     x_value_x_in = np.delete(x_value_x[0, :], instruments)[None, :]
     A_eq[x, x] = model_ya.predict_proba(x_value_x_in)[0][1] 
     A_eq[x, x_space + x] = model_ya.predict_proba(x_value_x_in)[0][0]
     b_eq[x] = model_a.predict_proba(x_value_x)[0][1]

   # Inequality constraints: 
   # P(A = 1 | X[x_pos] = x_value[x_pos], X[\x_pos] = x_value[\x_pos], Y_a = 1) * 
   #    P(Y_a = 1 | X[x_pos] = x_value[x_pos], X[\x_pos] = x_value[\x_pos]) /
   #        P(A = 1 | X[x_pos] = x_value[x_pos], X[\x_pos] = x_value[\x_pos])) -   
   # P(A = 1 | X[x_pos] = x, X[\x_pos] = x_value[\x_pos], Y_a = 1) * 
   #    P(Y_a = 1 | X[x_pos] = x, X[\x_pos] = x_value[\x_pos]) /
   #        P(A = 1 | X[x_pos] = x, X[\x_pos] = x_value[\x_pos])) <= eps
   # 
   # Then, the same constraint, but now >= -eps. 
   #
   # Then, a repeat of the above.

   A_ineq = np.zeros([4 * (x_space - 1), 2 * x_space])
   b_ineq = np.zeros(4 * (x_space - 1))
   ineq_count = 0

   for x in range(x_space):

      if xg == x:
        continue

      x_value_x = x_value.copy()
      x_value_x[0, x_pos] = x
      x_value_x_in = np.delete(x_value_x[0, :], instruments)[None, :]
      p_y0_x_value, p_y1_x_value = model_ya.predict_proba(x_value_in)[0][0], model_ya.predict_proba(x_value_in)[0][1]
      p_y0_x_value_x, p_y1_x_value_x = model_ya.predict_proba(x_value_x_in)[0][0], model_ya.predict_proba(x_value_x_in)[0][1]
      p_a1_x_value, p_a1_x_value_x = model_a.predict_proba(x_value)[0][1], model_a.predict_proba(x_value_x)[0][1]
   
      A_ineq[ineq_count, xg] = p_y1_x_value * p_a1_x_value_x
      A_ineq[ineq_count, x] = -p_y1_x_value_x * p_a1_x_value
      b_ineq[ineq_count] = eps * p_a1_x_value * p_a1_x_value_x
      ineq_count = ineq_count + 1 
   
      A_ineq[ineq_count, xg] = -p_y1_x_value * p_a1_x_value_x
      A_ineq[ineq_count, x] = p_y1_x_value_x * p_a1_x_value
      b_ineq[ineq_count] = eps * p_a1_x_value * p_a1_x_value_x
      ineq_count = ineq_count + 1 

      A_ineq[ineq_count, x_space + xg] = p_y0_x_value * p_a1_x_value_x
      A_ineq[ineq_count, x_space + x] = -p_y0_x_value_x * p_a1_x_value
      b_ineq[ineq_count] = eps * p_a1_x_value * p_a1_x_value_x
      ineq_count = ineq_count + 1

      A_ineq[ineq_count, x_space + xg] = -p_y0_x_value * p_a1_x_value_x
      A_ineq[ineq_count, x_space + x] = p_y0_x_value_x * p_a1_x_value
      b_ineq[ineq_count] = eps * p_a1_x_value * p_a1_x_value_x
      ineq_count = ineq_count + 1

   # Solve program

   if test_values is not None:
     lhs = A_ineq @ test_values
     for i in range(len(b_ineq)):
       print(lhs[i], ' <= ', b_ineq[i], end="")
       if lhs[i] <= b_ineq[i]:
         print(' yes')
       else:
         print(' NO')
     print('Maximum equality violation = ', np.max(np.abs(A_eq @ test_values - b_eq)))
     print('Maximum inequality gap = ', np.max(A_ineq @ test_values - b_ineq))
     return None, None

   bounds_matrix = np.zeros([2 * x_space, 2])
   bounds_matrix[:, 1] = 1
   bounds = [tuple(row) for row in bounds_matrix]
   c = np.zeros(2 * x_space)

   c[xg] = 1
   result = linprog(c, A_eq=A_eq, b_eq=b_eq, A_ub=A_ineq, b_ub=b_ineq, bounds=bounds)
   l = result.fun
   l_theta = result.x

   c[xg] = -1
   result = linprog(c, A_eq=A_eq, b_eq=b_eq, A_ub=A_ineq, b_ub=b_ineq, bounds=bounds)
   u = result.fun
   u_theta = result.x
   if u is not None:
     u = -u

   return l, u, l_theta, u_theta

def bound_finding(x_spaces, eps, train_idx, test_idx, dat_obs, instruments,
                  models_a, models_y1, models_y0, models_y1o, models_y0o):

  """
  Given a model for P(Y_a = 1 | X = `x_value`) and P(A = 1 | X = `x_value`), this returns lower and upper bounds
  on P(A = 1 | X = `x_value`, Y_a = 1). (Notice that "a" in {0, 1} is implicit here, given indirectly by 
  P(Y_a = 1 | X = `x_value') model). It does so by making use of X[`x_pos`] as an instrument, and assuming 
  particular constraints bounded by `eps`.

  Parameters:
  -----------

  x_spaces : an array of integers indicating that X[i] takes values in 0, 1, 2, ..., x_spaces[i] - 1
  eps      : corresponding bound to define constraints
  train_idx : an array indicating which elements of dat_obs are to be used as training set
  test_idx : an array indicating which elements of dat_obs are to be used as test set
  dat_obs  : a list where dat_obs[i] is an observabtional dataset
  instruments : a list where instruments[i] are the positions of the covariates which are instruments
                in test set i
  models_a  : a list wheere models_a[i] is model that returns P(A = 1 | X = x), a xgboost tree, 
              for environment i
  models_y1 : analogous, but models_y1[i] is a model that returns P(Y_1 | X = x)
  models_y0 : analogous, but models_y0[i] is a model that returns P(Y_0 | X = x)
  models_y1o : analogous, but models_y1[i] is a model that returns P(Y | A = 1, X = x)
  models_y0o : analogous, but models_y1[i] is a model that returns P(Y | A = 0, X = x)

  x_value  : instance of covariates where we are evaluating the quantity of interest
  x_pos    : position in the covariate vector of the variable which is the inferred instrument

  Returns:
  --------
    
  lower_bounds_1, upper_bounds_1 : bounds for P(Y_1 = 1 | X = x) in each test environment
  lower_bounds_0, upper_bounds_0 : bounds for P(Y_0 = 1 | X = x) in each test environment
  """

  def combine_bounds(lower_bounds, upper_bounds):
    """
    Combination rule for bounds extracted from training environments. Using here the simplest, most
    conservative rule, which is likely be too conservative: it is enough to have one bad environment
    where the selected instrument is weakly conditionally associated with treatment to 
    get trivial bounds.
    """
    if len(lower_bounds) > 0:
      l = min(lower_bounds)
    else:
      l = 0
    if len(upper_bounds) > 0:
      u = max(upper_bounds)
    else:
      u = 1
    return l, u

  eps = 0.01                         # Hyperparameter for bounding procedure
  num_test = len(test_idx)           # Number of test sets

  cw_lower_bounds_1 = [None] * num_test # Array that stores lower bounds for P(A = 1 | X = x, Y_1 = 1, F = idle) across test sets
  cw_lower_bounds_0 = [None] * num_test # Array that stores lower bounds for P(A = 1 | X = x, Y_1 = 0, F = idle) across test sets
  cw_upper_bounds_1 = [None] * num_test # Array that stores upper bounds for P(A = 1 | X = x, Y_1 = 1, F = idle) across test sets
  cw_upper_bounds_0 = [None] * num_test # Array that stores upper bounds for P(A = 1 | X = x, Y_1 = 0, F = idle) across test sets
  lower_bounds_1 = [None] * num_test    # Array that stores lower bounds for P(Y(A = 1) | X = x) across test sets
  lower_bounds_0 = [None] * num_test    # Array that stores lower bounds for P(Y(A = 0) | X = x) across test sets
  upper_bounds_1 = [None] * num_test    # Array that stores upper bounds for P(Y(A = 1) | X = x) across test sets
  upper_bounds_0 = [None] * num_test    # Array that stores upper bounds for P(Y(A = 0) | X = x) across test sets

  for i, test_i in enumerate(test_idx): # Go through each test set

    n_i, n_j = dat_obs[test_i].shape[0], len(instruments[test_i])

    cw_lower_bounds1_j = np.zeros([n_i, n_j]) # P(A = 1 | X = x, Y_1 = 1, F = idle) bounds stratified by instrument
    cw_upper_bounds1_j = np.ones([n_i, n_j])
    cw_lower_bounds0_j = np.zeros([n_i, n_j])
    cw_upper_bounds0_j = np.ones([n_i, n_j])
    cw_lower_bounds_1[i] = np.zeros(n_i)      # P(A = 1 | X = x, Y_1 = 1, F = idle) bounds aggregated through all instruments
    cw_lower_bounds_0[i] = np.zeros(n_i)
    cw_upper_bounds_1[i] = np.zeros(n_i)
    cw_upper_bounds_0[i] = np.zeros(n_i)                           
    lower_bounds_1[i] = np.zeros(n_i)         # P(Y(A = *) | X = x)  bounds in this test set
    lower_bounds_0[i] = np.zeros(n_i)
    upper_bounds_1[i] = np.zeros(n_i)
    upper_bounds_0[i] = np.zeros(n_i)

    print('Getting bounds, test set', i)
    for m in tqdm(range(n_i)): # Go through each data point in test set i

      # Extract corresponding covariate values at which we will bound causal response
      x_value = dat_obs[test_i][x_idx].iloc[[m], :].to_numpy()

      for j, x_pos in enumerate(instruments[test_i]): # For each instrument from test set 
        
        ls1, us1 = [], []         # Lower and upper bounds of cross-world P(A = 1 | X = x_value, Y_* = 1, F = idle)
        ls0, us0 = [], []         # Lower and upper bounds of cross-world P(A = 1 | X = x_value, Y_* = 0, F = idle)
        x_space = x_spaces[x_pos] # Number of categories of instrument x_pos
        
        for v in train_idx: # For each training environment
          if x_pos in instruments[v]: # Only look at those training sets where that variable is also a instrument
            # Lower and upper bounds of cross-world P(A = 1 | X = x_value, Y_* = 1, F = idle) at environment v
            l1, u1, _, _ = bound_po(models_y1[v], models_a[v], x_value, x_pos, x_space, instruments[v], eps)
            # Lower and upper bounds of cross-world P(A = 1 | X = x_value, Y_* = 1, F = idle) at environment v
            l0, u0, _, _ = bound_po(models_y0[v], models_a[v], x_value, x_pos, x_space, instruments[v], eps)
            if l1 is not None: # Only add if problem is feasible (it will be infeasiable if dependence measure is not weak enough)
              ls1.append(l1)
              us1.append(u1)
              ls0.append(l0)
              us0.append(u0)

        # Transfer rule: combine bounds across environments for test point m as given by j-th instrument x_pos
        cw_lower_bounds1_j[m, j], cw_upper_bounds1_j[m, j] = combine_bounds(ls1, us1)
        cw_lower_bounds0_j[m, j], cw_upper_bounds0_j[m, j] = combine_bounds(ls0, us0)
      
      # Now combine bounds across all instruments by getting maximum lower bound and minimum lower bound
      cw_lower_bounds_1[i][m] = max(cw_lower_bounds1_j[m, :])
      cw_upper_bounds_1[i][m] = min(cw_upper_bounds1_j[m, :])
      cw_lower_bounds_0[i][m] = max(cw_lower_bounds0_j[m, :])
      cw_upper_bounds_0[i][m] = min(cw_upper_bounds0_j[m, :])

      # Final step: now that we have bounds on cross-world P(A = 1 | X = x_value, Y_* = 1, F = idle),
      # propagate these to get bounds on P(Y(A = 0) = 1 | X = x_value) and P(Y(A = 1) = 1 | X = x_value)
      x_value_sel = np.delete(x_value[0, :], instruments[test_i]).reshape(1, len(x_idx) - len(instruments[test_i]))
      p_y_1 = models_y1o[i].predict_proba(x_value_sel)[0][1] # P(Y = 1 | A = 1, X = x_value)
      p_a_1 = models_a[i].predict_proba(x_value)[0][1]   # P(A = 1 | X = x_value)
      p_y_0 = models_y0o[i].predict_proba(x_value_sel)[0][1] # P(Y = 1 | A = 0, X = x_value)
      p_a_0 = models_a[i].predict_proba(x_value)[0][0]   # P(A = 0 | X = x_value)
      lower_bounds_1[i][m] = p_y_1 * p_a_1 / cw_upper_bounds_1[i][m]
      lower_bounds_0[i][m] = p_y_0 * p_a_0 / cw_upper_bounds_0[i][m]
      if cw_lower_bounds_1[i][m] > 0:
        upper_bounds_1[i][m] = p_y_1 * p_a_1 / cw_lower_bounds_1[i][m]
        if upper_bounds_1[i][m] > 1:
          upper_bounds_1[i][m] = 1
      else:
        upper_bounds_1[i][m] = 1
      if cw_lower_bounds_0[i][m] > 0:
        upper_bounds_0[i][m] = p_y_0 * p_a_0 / cw_lower_bounds_0[i][m]
        if upper_bounds_0[i][m] > 1:
          upper_bounds_0[i][m] = 1
      else:
        upper_bounds_0[i][m] = 1

  return lower_bounds_1, upper_bounds_1, lower_bounds_0, upper_bounds_0


-------
## Example of a Run

The following is a full end-to-end run of the algorithm, making use of a fully synthetic simulator.

In [6]:
# Simulation study: generate synthetic models and datasets
# This uses a very bare-bones simulator that should be substituted.
#
# In all that follows:
#  - we will use A to denote binary treatment variable
#  - we will use X to denote (categorical) covariates
#  - we will use Y to denote binary outcome variables
#
# Notice that in the manuscript, we used X to denote treatments and Z to denote covariates.
#
# The main data structure used here to store data are Pandas dataframes, where categorical
# variables are represented explicitly as categories. No one-hot encoding is used, it's one
# column per categorical variable.

k = 10         # Number of datasets
n_obs = 200    # Sample size for observational datasets
n_rct = 200    # Sample size for RCTs
num_x = 20     # Number of covariates
p_sparse = 0.2 # Probability of dropping a covariate in the outcome model

# Simulate true models (using a very poorly tested simulator as a placeholder only).
# Simulated models are all binary with some edges from covariates to outcome removed.
# The conditional distribution of each variable given parents is a logistic regression model.
#
# Notice though that we define two separate logistic regression models for the outcome stratified by treatment.
# That is, P(Y(A = 1) | X = x) and P(Y(A = 0) | X = x) are two distinctive models. So P(Y | A = a, X = x) is 
# non-linear on A.
 
print('Simulating models and data... ', end="")
true_models, dat_obs, dat_rct, true_cates = simulate_plain_data_collection(k, n_obs, n_rct, num_x, p_sparse)
train_idx = range(k - 2)  # Environments for training
test_idx = [k - 2, k - 1] # Environments for testing
print('Done!')

# Here, x_idx refers to a list of strings denoting column names for covariates, a_idx is a string denoting
# column name for treatment, y_idx is a string denoting the column name for the outcome
x_idx, a_idx, y_idx = true_models[0].x_idx, true_models[0].a_idx, true_models[0].y_idx

Simulating models and data... Done!


In [ ]:
# This block discover instruments in each training set. It is currently working very poorly, based
# on a vary bare-bones sparse logistic regression algorithm

print('Discovering instruments...')
instruments = [None] * k
for i in tqdm(range(k)):
  if true_models is not None:
    instruments[i] = np.where(true_models[0].coeff_y1 == 0)[0] - 1
  else:
    # This is not working well at all, which is the reason currently we are relying on true_models
    instruments[i] = search_adjustment(dat_obs[i], x_idx, a_idx, y_idx)
    
print('Learning predictive models...') # Currently, this is slow (xgboost with cross-validation). Maybe someone can improve it?
models_a, models_y1, models_y0, models_y1o, models_y0o = [None] * k, [None] * k, [None] * k, [None] * k, [None] * k
for i in tqdm(range(k)): 
  # models_a[i] is the propensity score for each environment (training and test)
  # models_y1[i] is a model for P(Y(A = 1) = y | X = x) directly from RCT data (assumes RCT data available to all datasets)
  # models_y0[i] is a model for P(Y(A = 0) = y | X = x) directly from RCT data
  # models_y1o[i] is a model for P(Y = y | A = 1, X = x) from observational data
  # models_y0o[i] is a model for P(Y = y | A = 0, X = x) from observational data
  models_a[i], models_y1[i], models_y0[i], models_y1o[i], models_y0o[i] = \
    get_predictive_model(dat_obs[i], dat_rct[i], x_idx, a_idx, y_idx, instruments[i])

Discovering instruments...


100%|██████████| 10/10 [00:00<00:00, 70374.23it/s]


Learning predictive models...


  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
# This blocks return lower and upper bounds of P(Y_1 | X = x) and P(Y_0 | X = x) for 
# all test sets

eps = 0.01 # Hyperparameter regulating bounding procedure
x_spaces = true_models[0].x_space  # Array that stores number of categories for each covariate

lower_bounds_1, upper_bounds_1, lower_bounds_0, upper_bounds_0 = \
  bound_finding(x_spaces, eps, train_idx, test_idx, dat_obs, instruments,
                models_a, models_y1, models_y0, models_y1o, models_y0o)


In [ ]:
# Comparison:
#
# Let's pretend that regime "off" if P(Y | A = 1, X = x) and that regime "on" is P(Y(A = 1) | X = x)
# We will then compare the true P(Y(A = 1) = 1 | X = x) and the bounds, plus we will compare the "off" regime and "on"
# regime, indicating when the lower bound of the "on" regime is higher that the corresponding outcome on 
# the "off" regime.

num_test = len(test_idx)           # Number of test sets

p_off = [None] * num_test
p_gold = [None] * num_test 
hit_ratio = np.zeros(num_test)
informative_ratio = np.zeros(num_test)

for i, test_i in enumerate(test_idx): # Go through each test set
  x_idx_sel = [x for w, x in enumerate(x_idx) if w not in instruments[test_idx[0]]]
  p_off[i] = models_y1o[i].predict_proba(dat_obs[test_i][x_idx_sel])[:, 1]
  p_gold[i] = models_y1[i].predict_proba(dat_obs[test_i][x_idx_sel])[:, 1]
  hit_ratio[i] = np.mean((p_gold[i] <= upper_bounds_1[i]) * (p_gold[i] >= lower_bounds_1[i]))
  informative_ratio[i] = np.mean(p_off[i] <= lower_bounds_1[i])
  print('Interval coverage:', hit_ratio[i], 'for test set', i)
  print('Informativeness ratio:', informative_ratio[i], 'for test set', i)


In [ ]:
np.mean((p_gold[i] <= upper_bounds_1[i]) * (p_gold[i] >= lower_bounds_1[i]))

In [ ]:
p_gold[i]